<a href="https://colab.research.google.com/github/JCaballerot/Recommender_Systems/blob/main/XGBoost_Recommender/Book_Crossing_XGB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<h1 align=center><font size = 5> K-Nearest Neighbors
 Recommender</font></h1>

---

<center>
  <img src="https://storage.googleapis.com/kaggle-datasets-images/1661575/2726067/684ac0c4c14cb46d1047ccb620b45cac/dataset-cover.jpg?t=2021-10-21-03-18-09" width="800" height="300">
</center>


## Objetivo de este Notebook

1. Cargar y preprocesar un Dataset.
2. Realizar un sistema de recomendación basado en KNN.
3. Comprobar el performance del sistema.

## Tabla de Contenidos

<div class="alert alert-block alert-info" style="margin-top: 20px">

<font size = 3>
    
1. <a href="#item31">Contexto</a>  
2. <a href="#item32">Descargar y preparar el Dataset</a>  
6. <a href="#item34">Entrenamiento del modelo</a>  
6. <a href="#item34">Validación del modelo</a>  

</font>
</div>

### 1. Contexto


El conjunto de datos "Book-Crossing" (también conocido como BX) es una colección de datos relacionados con libros y reseñas de libros. Este conjunto de datos se centra en la interacción de los usuarios con libros y sus calificaciones, y es ampliamente utilizado en aplicaciones de sistemas de recomendación.



<b>Descripción de datos</b>

---

El conjunto de datos Book-Crossing contiene información sobre:

* <b>Libros:</b> Información sobre los libros, incluyendo su título, autor y año de publicación.

* <b>Usuarios:</b> Perfiles de los usuarios que interactúan con los libros, incluyendo su ID y ubicación.

* <b>Calificaciones:</b> Calificaciones numéricas que los usuarios asignan a los libros que han leído.

El conjunto de datos puede ser utilizado para varios propósitos, como la construcción de sistemas de recomendación de libros, el análisis de patrones de lectura y preferencias de los usuarios, y la investigación en el campo de la minería de datos y la inteligencia artificial.

---



<strong>Puede consultar este [link](https://www.kaggle.com/datasets/syedjaferk/book-crossing-dataset) para leer más sobre la fuente de datos Book Crossing.</strong>


### 2. Descargar y preparar Dataset

In [ ]:
# Download Book-Crossing Dataset
!curl -o dataset.zip "http://www2.informatik.uni-freiburg.de/~cziegler/BX/BX-CSV-Dump.zip"
!unzip dataset.zip
!ls -la

In [75]:
# Principales librerías
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore") # Turn off warnings


In [ ]:
ratings = pd.read_csv("BX-Book-Ratings.csv", sep=";", encoding="ISO-8859-1")
books   = pd.read_csv("BX-Books.csv",        sep=";", encoding="ISO-8859-1", error_bad_lines=False)
users   = pd.read_csv("BX-Users.csv",        sep=";", encoding="ISO-8859-1")

In [ ]:
users.head()

In [ ]:
books.head()

<b>Calificaciones explícitas</b>: Están expresadas en una escala del 1-10 (más alta) y representan una calificación explícita por parte del usuario.

<b>Calificaciones implícitas</b>: Son expresadas por un 0, indicando que no hay una calificación explícita. En el contexto de este dataset, una calificación de 0 indica una interacción implícita con el libro (por ejemplo, el usuario lo compró o leyó), pero no proporciona una calificación explícita del contenido.

In [ ]:
ratings.head()

In [ ]:
print("  Users: {} \n  Books: {}\n  Ratings: {}".format(len(users), len(books), len(ratings)))


In [81]:
users.columns = users.columns.str.lower().str.replace('-', '_')
books.columns = books.columns.str.lower().str.replace('-', '_')
ratings.columns = ratings.columns.str.lower().str.replace('-', '_')

### 3. Uniendo data

In [183]:
ratings = ratings[ratings.book_rating > 0]

In [ ]:
# Cruzamos las bases de datos para  obtener una tabla única

data = pd.merge(ratings, users, on = 'user_id', how = 'left')
data = pd.merge(data,    books, on = 'isbn', how = 'left')
data.drop(columns = ['image_url_s', 'image_url_m', 'image_url_l'], inplace = True)

data.head()

In [ ]:
# Estilo de Seaborn
sns.set(style="whitegrid")
# figura y eje
plt.figure(figsize=(6, 3))
sns.histplot(data.book_rating, bins=30, kde=False, color="skyblue")

In [186]:
#tratando información del año de publicación
data.year_of_publication = pd.to_numeric(data.year_of_publication, errors='coerce')


In [187]:
# Ejemplo de remoción de outliers
lower_threshold = 1964
upper_threshold = 2004

data = data[(data['year_of_publication'] >= lower_threshold) & (data['year_of_publication'] <= upper_threshold)]
data.year_of_publication = data.year_of_publication.astype(int)

In [188]:
#Creando antiguedad del libro
data['antiguedad'] = 2023 - data.year_of_publication

In [ ]:
# Estilo de Seaborn
sns.set(style="whitegrid")

# figura y eje
plt.figure(figsize=(6, 3))

# histograma
sns.histplot(data.antiguedad, bins=30, kde=False, color="skyblue")

# título y etiquetas a los ejes
plt.title('Distribución de antiguedad', fontsize=12)
plt.xlabel('Antiguedad', fontsize=10)
plt.ylabel('Frecuencia', fontsize=10)

# Muestra el histograma
plt.show()

In [ ]:
books_list = data.groupby('book_title')['user_id'].count().reset_index()
books_list.sort_values(by = 'user_id', ascending = False, inplace = True)

print(f"{len(books_list)} libros diferentes, nos quedaremos con los más populares para no saturar nuestro Recsys")

In [191]:
# Calculamos los libros más populares
pop_books = books_list[:500].book_title.tolist()

In [192]:
data_v2 = data[data.book_title.isin(pop_books)]

In [ ]:
data_v2.head()

Dicotomizaremos la variable objetivo para que el modelo aprenda la probabilidad de que el cliente tenga afinidad con el libro. Esta estrategia es bastante utilizada en las aplicaciones de Recsys pero no olvidemos que también se puede apuntar a predecir directamente el rating del cliente.

In [203]:
data_v2['target'] = data_v2.book_rating.apply(lambda x: 1 if x > 7 else 0)

In [ ]:
data_v2.head()

In [ ]:
# figura y eje
plt.figure(figsize=(6, 3))
# Analizando el target
sns.countplot(x='target', data = data_v2, palette = 'hls')
plt.title('¿La data presenta desbalance?', fontsize=12)


### 4. Muestreo de datos

In [204]:
# Muestreo de data
from sklearn.model_selection import train_test_split

train, test = train_test_split(data_v2,
                               stratify = data_v2.target, # Recuerda estratificar para evitar sesgos durante el muestreo
                               train_size = 0.6,
                               random_state = 123)

watch, test = train_test_split(test,
                               stratify = test.target, # Recuerda estratificar para evitar sesgos durante el muestreo
                               train_size = 0.5,
                               random_state = 123)

# El muestreo puede hacerse por cliente o por enmascaramiento como en anteriores ejercicios.

### 4. Tratamiento de variables

Variable de locacion

In [ ]:
train.head()

In [ ]:
temp = train.groupby('location')['user_id'].count().reset_index()
temp.sort_values(by = 'user_id', ascending = False)

In [206]:
# Función para extraer los n últimos elementos y unirlos con ','
def extract_last_n(location, n):
    parts = location.split(', ')
    return ', '.join(parts[-n:])

# Generar agregaciones
train['location_level2'] = train['location'].apply(lambda x: extract_last_n(x, 2))
train['location_level3'] = train['location'].apply(lambda x: extract_last_n(x, 1))

test['location_level2'] = test['location'].apply(lambda x: extract_last_n(x, 2))
test['location_level3'] = test['location'].apply(lambda x: extract_last_n(x, 1))

watch['location_level2'] = watch['location'].apply(lambda x: extract_last_n(x, 2))
watch['location_level3'] = watch['location'].apply(lambda x: extract_last_n(x, 1))

In [ ]:
train.head()

In [ ]:
temp = train.groupby('location_level2')['user_id'].count().reset_index()
temp = temp[temp.user_id > 30]
temp.sort_values(by = 'user_id', ascending = False)

In [ ]:
temp = train.groupby('location_level3')['user_id'].count().reset_index()
temp = temp[temp.user_id > 30]
temp.sort_values(by = 'user_id', ascending = False)

In [208]:
# Creando variable mixta de locacion
train['location_f'] = train.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)
test['location_f']  = test.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)
watch['location_f'] = watch.apply(lambda row: row['location_level2'] if row['location_level3'] == 'usa' else row['location_level3'], axis=1)


In [ ]:
train.head()

**Encoding**

El encoding de variables categóricas convierte las categorías de texto en números de una manera que puede ser utilizada de manera eficiente por los algoritmos de machine learning.


In [ ]:
train.head()

In [209]:
catergory_features = ['book_title', 'book_author', 'publisher', 'location_f']

In [143]:
%%capture
!pip3 install category_encoders

In [179]:
# Aplicando category encoders
from category_encoders import TargetEncoder

encoder = TargetEncoder(handle_unknown = 'infrequent_if_exist',
                        handle_missing = 'value',
                        min_samples_leaf = 30)

encoder.fit(train[catergory_features].astype('category'), train['target'])


TargetEncoder(cols=['book_title', 'book_author', 'publisher', 'location_f'],
              handle_unknown='infrequent_if_exist', min_samples_leaf=30)

In [180]:
# Aplicando transformaciones sobre  variables

train[[x + '_coded' for x in catergory_features]] = encoder.transform(train[catergory_features].astype('category'))
test[[x + '_coded' for x in catergory_features]]  = encoder.transform(test[catergory_features].astype('category'))
watch[[x + '_coded' for x in catergory_features]] = encoder.transform(watch[catergory_features].astype('category'))


In [ ]:
train.head()

### 5. Modelamiento

In [154]:
features = ['age', 'antiguedad', 'book_title_coded', 'book_author_coded', 'publisher_coded', 'location_f_coded']

In [162]:
import xgboost as xgb
from sklearn.metrics import *


In [172]:
# Definimos los parámetros para el Grid Search

param_grid = {'objective': ['binary:logistic'],
              'learning_rate': [0.01, 0.05, 0.1],
              'max_depth': [3, 5, 7],
              'colsample_bytree': [0.7, 1],
              'subsample': [0.7, 1]}


In [181]:
%%time
from sklearn.model_selection import GridSearchCV

# Crear clasificador
xgBoost = xgb.XGBClassifier(use_label_encoder=False, n_estimators = 500)


# Crear objeto GridSearchCV
grid_search = GridSearchCV(xgBoost,
                           param_grid,
                           scoring = make_scorer(auc),
                           cv = 3,  # Número de folds en la validación cruzada
                           verbose = 2,  # Verbosidad del output
                           n_jobs = -1  # Uso de todos los núcleos disponibles
                          )

# Realizar búsqueda de parámetros
grid_search.fit(train[features],
                train.target,
                early_stopping_rounds = 10,
                eval_metric = "auc",
                eval_set=[(watch[features], watch.target)],
                verbose = True)



Fitting 3 folds for each of 36 candidates, totalling 108 fits
[0]	validation_0-auc:0.60265
[1]	validation_0-auc:0.60641
[2]	validation_0-auc:0.61658
[3]	validation_0-auc:0.61403
[4]	validation_0-auc:0.61763
[5]	validation_0-auc:0.61786
[6]	validation_0-auc:0.61760
[7]	validation_0-auc:0.61820
[8]	validation_0-auc:0.61780
[9]	validation_0-auc:0.61749
[10]	validation_0-auc:0.61733
[11]	validation_0-auc:0.61721
[12]	validation_0-auc:0.61699
[13]	validation_0-auc:0.61696
[14]	validation_0-auc:0.61695
[15]	validation_0-auc:0.61719
[16]	validation_0-auc:0.61712
CPU times: user 2.57 s, sys: 447 ms, total: 3.01 s
Wall time: 34 s


GridSearchCV(cv=3,
             estimator=XGBClassifier(base_score=None, booster=None,
                                     callbacks=None, colsample_bylevel=None,
                                     colsample_bynode=None,
                                     colsample_bytree=None, device=None,
                                     early_stopping_rounds=None,
                                     enable_categorical=False, eval_metric=None,
                                     feature_types=None, gamma=None,
                                     grow_policy=None, importance_type=None,
                                     interaction_constraints=None,
                                     learning_rate=None,...
                                     max_leaves=None, min_child_weight=None,
                                     missing=nan, monotone_constraints=None,
                                     multi_strategy=None, n_estimators=200,
                                     n_jobs=None, num_parallel_tree=None,
                                     random_state=None, ...),
             n_jobs=-1,
             param_grid={'colsample_bytree': [0.7, 1],
                         'learning_rate': [0.01, 0.05, 0.1],
                         'max_depth': [3, 5, 7],
                         'objective': ['binary:logistic'],
                         'subsample': [0.7, 1]},
             scoring=make_scorer(auc), verbose=2)

In [170]:
xgBoost = xgb.XGBClassifier(use_label_encoder=False, n_estimators = 200)
xgBoost.fit(train[features], train.target,
            early_stopping_rounds=10, eval_metric="auc",
                eval_set=[(test[features], test.target)], verbose=True)

[0]	validation_0-auc:0.61227
[1]	validation_0-auc:0.61553
[2]	validation_0-auc:0.61676
[3]	validation_0-auc:0.61766
[4]	validation_0-auc:0.61865
[5]	validation_0-auc:0.61911
[6]	validation_0-auc:0.61977
[7]	validation_0-auc:0.62048
[8]	validation_0-auc:0.62088
[9]	validation_0-auc:0.62135
[10]	validation_0-auc:0.62185
[11]	validation_0-auc:0.62217
[12]	validation_0-auc:0.62237
[13]	validation_0-auc:0.62269
[14]	validation_0-auc:0.62305
[15]	validation_0-auc:0.62346
[16]	validation_0-auc:0.62357
[17]	validation_0-auc:0.62360
[18]	validation_0-auc:0.62445
[19]	validation_0-auc:0.62533
[20]	validation_0-auc:0.62542
[21]	validation_0-auc:0.62653
[22]	validation_0-auc:0.62664
[23]	validation_0-auc:0.62694
[24]	validation_0-auc:0.62690
[25]	validation_0-auc:0.62688
[26]	validation_0-auc:0.62712
[27]	validation_0-auc:0.62796
[28]	validation_0-auc:0.62955
[29]	validation_0-auc:0.62964
[30]	validation_0-auc:0.62975
[31]	validation_0-auc:0.62966
[32]	validation_0-auc:0.62979
[33]	validation_0-au

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=200, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)

In [ ]:
# Definimos el Grid Search con validación cruzada (cross-validation)

grid_search = GridSearchCV(estimator = model,
                           param_grid = param_grid,
                           cv = 3,
                           scoring = 'auc', verbose = 1, n_jobs = -1)

# Ajustamos el Grid Search al conjunto de datos
grid_search.fit(X, y)

# Imprimimos los mejores parámetros encontrados
print("Mejores parámetros encontrados: ", grid_search.best_params_)

# 3. K-Nearest Neighbors

K-Nearest Neighbors (KNN) es un algoritmo de machine learning que también se puede utilizar en sistemas de recomendación. La idea detrás del uso de KNN en sistemas de recomendación es encontrar usuarios o elementos similares en función de sus calificaciones o comportamientos previos y utilizar esa similitud para hacer recomendaciones.



* <b>Matriz de usuario-elemento:</b> Se crea una matriz que representa las calificaciones de los usuarios para los elementos. Cada fila de la matriz representa un usuario, y cada columna representa un elemento. Los valores de la matriz son las calificaciones dadas por los usuarios a los elementos.

* <b>Recomendación:</b>  Se utilizan las calificaciones de los K usuarios o elementos más cercanos para generar recomendaciones para el usuario o elemento en cuestión. Esto se puede hacer de varias maneras, como calcular un promedio ponderado de las calificaciones de los vecinos o identificar los elementos mejor calificados por los vecinos.

* <b> Evaluación:</b>  Se evalúa el rendimiento del sistema de recomendación utilizando métricas como RMSE (Root Mean Squared Error) o MAE (Mean Absolute Error) en un conjunto de datos de prueba para medir cuán precisas son las recomendaciones.

---


### 3.1. Muestreo de datos


El conjunto de datos en machine learning se divide típicamente en dos partes: el conjunto de entrenamiento (train) y el conjunto de prueba (test). Estas divisiones se utilizan para entrenar y evaluar los modelos.



<b>Train:</b> El conjunto de entrenamiento se utiliza para entrenar el modelo de machine learning. Es aquí donde el modelo "aprende" los patrones y relaciones en los datos para poder hacer predicciones o clasificaciones.

<b>Test:</b> El conjunto de prueba se utiliza para evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento. Es una medida objetiva de la capacidad del modelo para generalizar y realizar predicciones precisas en nuevos datos.

In [ ]:
# Muestreo
#La función train_test_split de scikit-learn se utiliza para dividir un conjunto de datos en subconjuntos de train y test.
from sklearn.model_selection import train_test_split

train, test = train_test_split(df_unified_filtered, # Base de datos
                               stratify = df_unified_filtered.book_title,
                               train_size = 0.7, # Especificar el tamaño de train/test
                               random_state = 123) # Semilla aleatoria



In [ ]:
train.book_title.value_counts()

The Lovely Bones: A Novel                                           319
Wild Animus                                                         317
The Da Vinci Code                                                   247
The Secret Life of Bees                                             186
Bridget Jones's Diary                                               180
Harry Potter and the Chamber of Secrets (Book 2)                    169
The Nanny Diaries: A Novel                                          169
Life of Pi                                                          169
Angels &amp; Demons                                                 168
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))    152
Name: book_title, dtype: int64

In [ ]:
test.book_title.value_counts()

The Lovely Bones: A Novel                                           137
Wild Animus                                                         136
The Da Vinci Code                                                   106
The Secret Life of Bees                                              80
Bridget Jones's Diary                                                77
The Nanny Diaries: A Novel                                           73
Angels &amp; Demons                                                  72
Life of Pi                                                           72
Harry Potter and the Chamber of Secrets (Book 2)                     72
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))     66
Name: book_title, dtype: int64

In [ ]:
# Crear una matriz pivot para el conjunto de entrenamiento
pivot_table_entrenamiento = train.pivot(index='user_id', columns='isbn', values='book_rating').fillna(0)

# Crear una matriz pivot para el conjunto de prueba
pivot_table_prueba = test.pivot(index='user_id', columns='isbn', values='book_rating').fillna(0)


In [ ]:
# Asegurémonos de que las columnas sean las mismas en ambos conjuntos
common_columns = pivot_table_entrenamiento.columns.intersection(pivot_table_prueba.columns)
pivot_table_entrenamiento = pivot_table_entrenamiento[common_columns]
pivot_table_prueba = pivot_table_prueba[common_columns]

### 3.2. KNN recommender


In [ ]:
from sklearn.neighbors import NearestNeighbors

In [ ]:

# Crear un modelo k-NN con 30 vecinos más cercanos
k = 30
model_knn = NearestNeighbors(n_neighbors=k, metric='cosine')
model_knn.fit(pivot_table_entrenamiento)



NearestNeighbors(metric='cosine', n_neighbors=30)

In [ ]:
pivot_table_entrenamiento.head()

isbn,014028009X,0141000198,0142001740,0151008116,0156027321,0312195516,0312278586,0312291639,0316666343,0330332775,...,059035342X,0670032379,0670880728,0670894605,0671027360,0739302043,0739307312,0743486226,0971880107,184195425X
user_id,,,,,,,,,,,,,,,,,,,,,
114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0
254,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
709,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
805,0.0,0.0,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
899,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0


In [ ]:
# Función para obtener recomendaciones para un usuario específico
def get_recommendations(user_ratings):
    distances, indices = model_knn.kneighbors([user_ratings], n_neighbors=k+1)  # +1 para excluir el propio usuario

    # Obtener los índices de los usuarios más cercanos (excluyendo el propio usuario)
    neighbor_indices = indices[0][1:]

    # Filtrar las calificaciones de los vecinos más cercanos
    neighbor_ratings = pivot_table_entrenamiento.iloc[neighbor_indices]

    # Calcular la puntuación promedio de los libros no calificados por el usuario
    book_scores = neighbor_ratings.mean()

    # Filtrar los libros que el usuario aún no ha calificado
    user_unrated_books = book_scores.index[~np.isnan(book_scores) & (user_ratings == 0)]

    # Ordenar los libros por puntuación promedio en orden descendente para obtener las recomendaciones
    recommendations = book_scores[user_unrated_books].sort_values(ascending=False)

    return recommendations

# Crear una tabla para almacenar las recomendaciones
recomendaciones_tabla = pd.DataFrame(columns=['user_id', 'isbn', 'puntuacion'])

# Para cada usuario en el conjunto de prueba, obtener sus recomendaciones
for user_id in pivot_table_prueba.index:
    user_ratings = pivot_table_prueba.loc[user_id].values
    recommendations = get_recommendations(user_ratings)
    # Agregar las recomendaciones a la tabla
    for isbn, score in recommendations.head(10).items():  # Tomar las 10 mejores recomendaciones
      recomendaciones_tabla = recomendaciones_tabla.append({'user_id': user_id, 'isbn': isbn, 'puntuacion': score}, ignore_index=True)



In [ ]:
recomendaciones_tabla_f = pd.merge(recomendaciones_tabla, books, on = 'isbn', how = 'left')

In [ ]:
recomendaciones_tabla_f.book_title.value_counts()

Bridget Jones's Diary                                               2249
The Secret Life of Bees                                             1588
Angels &amp; Demons                                                 1520
The Da Vinci Code                                                    818
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))     742
Wild Animus                                                          673
Harry Potter and the Chamber of Secrets (Book 2)                     420
Life of Pi                                                            91
The Lovely Bones: A Novel                                             81
The Nanny Diaries: A Novel                                            58
Name: book_title, dtype: int64

In [ ]:
test.book_title.value_counts()

The Lovely Bones: A Novel                                           137
Wild Animus                                                         136
The Da Vinci Code                                                   106
The Secret Life of Bees                                              80
Bridget Jones's Diary                                                77
The Nanny Diaries: A Novel                                           73
Angels &amp; Demons                                                  72
Life of Pi                                                           72
Harry Potter and the Chamber of Secrets (Book 2)                     72
Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))     66
Name: book_title, dtype: int64

In [ ]:
recomendaciones_tabla_f[recomendaciones_tabla_f.user_id == 32761].sort_values(by = 'puntuacion', ascending = False)

,user_id,isbn,puntuacion,book_title,book_author,year_of_publication,publisher
910,32761,0156027321,1.500000,Life of Pi,Yann Martel,2003,Harvest Books
911,32761,0316666343,1.166667,The Lovely Bones: A Novel,Alice Sebold,2002,"Little, Brown"
912,32761,0142001740,0.966667,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books
913,32761,0439064864,0.900000,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,1999,Scholastic
914,32761,0671027360,0.666667,Angels &amp; Demons,Dan Brown,2001,Pocket Star
915,32761,059035342X,0.666667,Harry Potter and the Sorcerer's Stone (Harry P...,J. K. Rowling,1999,Arthur A. Levine Books
916,32761,0439064872,0.666667,Harry Potter and the Chamber of Secrets (Book 2),J. K. Rowling,2000,Scholastic
917,32761,0971880107,0.600000,Wild Animus,Rich Shapero,2004,Too Far
918,32761,0312278586,0.600000,The Nanny Diaries: A Novel,Emma McLaughlin,2002,St. Martin's Press
919,32761,0385504209,0.533333,The Da Vinci Code,Dan Brown,2003,Doubleday


In [ ]:
test[test.user_id == 32761].head()

,user_id,age,isbn,book_rating,book_title
113020,32761,20.0,184195425X,8,Life of Pi


In [ ]:
recsys = pd.merge(recomendaciones_tabla_f, test, on = ['user_id', 'book_title'], how = 'left')
recsys = recsys[~recsys.book_rating.isnull()]
recsys[recsys.puntuacion > 0].head()

,user_id,isbn_x,puntuacion,book_title,book_author,year_of_publication,publisher,age,isbn_y,book_rating
137,3509,0671027360,0.366667,Angels &amp; Demons,Dan Brown,2001,Pocket Star,29.0,0743486226,8.0
225,6563,0142001740,0.633333,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books,31.0,0670032379,9.0
257,7125,0671027360,0.366667,Angels &amp; Demons,Dan Brown,2001,Pocket Star,28.0,0743486226,4.0
911,32761,0156027321,1.500000,Life of Pi,Yann Martel,2003,Harvest Books,20.0,184195425X,8.0
1276,43910,0142001740,0.333333,The Secret Life of Bees,Sue Monk Kidd,2003,Penguin Books,43.0,0670894605,8.0


---
## Gracias por completar este laboratorio!